In [ ]:
# Removes lint errors from VS Code
from typing import Dict, TYPE_CHECKING, Tuple, List

if TYPE_CHECKING:
    import kedro

    catalog: kedro.io.data_catalog.DataCatalog
    session: kedro.framework.session.session.KedroSession
    catalog: kedro.io.data_catalog.DataCatalog
    pipelines: Dict[str, kedro.pipeline.pipeline.Pipeline]

import pandas as pd

patients_wrk: pd.DataFrame = catalog.load("mimic_mm_core.wrk.patients")
admissions_wrk: pd.DataFrame = catalog.load("mimic_mm_core.wrk.admissions")
transfers_wrk: pd.DataFrame = catalog.load("mimic_mm_core.wrk.transfers")

patients_ref: pd.DataFrame = catalog.load("mimic_mm_core.ref.patients")
admissions_ref: pd.DataFrame = catalog.load("mimic_mm_core.ref.admissions")
transfers_ref: pd.DataFrame = catalog.load("mimic_mm_core.ref.transfers")

metadata: Dict = catalog.load("params:mimic_mm_core.metadata")


2000-01-01 00:00:00,000 - kedro.io.data_catalog - INFO - Loading data from `mimic_mm_core.wrk.patients` (ParquetDataSet)...
2000-01-01 00:00:00,000 - kedro.io.data_catalog - INFO - Loading data from `mimic_mm_core.wrk.admissions` (ParquetDataSet)...
2000-01-01 00:00:00,000 - kedro.io.data_catalog - INFO - Loading data from `mimic_mm_core.wrk.transfers` (ParquetDataSet)...
2000-01-01 00:00:00,000 - kedro.io.data_catalog - INFO - Loading data from `mimic_mm_core.ref.patients` (ParquetDataSet)...
2000-01-01 00:00:00,000 - kedro.io.data_catalog - INFO - Loading data from `mimic_mm_core.ref.admissions` (ParquetDataSet)...
2000-01-01 00:00:00,000 - kedro.io.data_catalog - INFO - Loading data from `mimic_mm_core.ref.transfers` (ParquetDataSet)...
2000-01-01 00:00:00,000 - kedro.io.data_catalog - INFO - Loading data from `params:mimic_mm_core.metadata` (MemoryDataSet)...


In [ ]:
tables = {
    "patients": [patients_wrk, patients_ref],
    "admissions": [admissions_wrk, admissions_ref],
    "transfers": [transfers_wrk, transfers_ref],
}


def gen_freq(a, b, cols, fillna=1e-6):
    if isinstance(cols, str):
        cols = [cols]

    a, b = a[cols].value_counts(), b[cols].value_counts()
    c = pd.concat([a, b], axis=1)
    c = c / c.sum()
    c = c.fillna(value=fillna)
    c = c / c.sum()
    return c.iloc[:, 0], c.iloc[:, 1]


In [ ]:
from scipy.stats import chisquare

res = []
for name, (a, b) in tables.items():
    for col in a.keys():
        if metadata["tables"][name]["fields"][col]["type"] == "categorical":
            k, j = gen_freq(a, b, col)
            chi, p = chisquare(k, j)
            # print(f"{name:12}.{col:20}: X^2={chi:3.3f} p={100*p:7.3f}%")
            res.append([name, col, chi, p])

res = pd.DataFrame(res, columns=["table", "col", "X^2", "p"])
res.set_index(keys=["table", "col"]).style.background_gradient(axis=0)


In [ ]:
from scipy.stats import ks_2samp

res = []
for name, (a, b) in tables.items():
    for col in a.keys():
        c, d = a[col].dropna(), b[col].dropna()
        match metadata["tables"][name]["fields"][col]["type"]:
            case "datetime" | "timespan":
                c, d = pd.to_numeric(c), pd.to_numeric(d)
            case "numerical":
                pass
            case _:
                continue
        ks, p = ks_2samp(c, d)
        res.append([name, col, ks, p])

res = pd.DataFrame(res, columns=["table", "col", "K-S", "p"])
res.set_index(keys=["table", "col"]).style.background_gradient(axis=0)


In [ ]:
def get_parent(table):
    match table:
        case 'patients':
            return None
        case 'admissions':
            return 'patients'
        case 'transfers':
            return 'admissions'
        case _:
            assert False, 'Table not found'

In [ ]:
from scipy.special import kl_div


def is_categorical(table, col):
    return metadata["tables"][table]["fields"][col]["type"] == "categorical"

res = []
for name, (a, b) in tables.items():
    cols = sorted(a.keys())
    for i, col_i in enumerate(cols):
        if not is_categorical(name, col_i):
            continue

        for col_j in cols[i + 1 :]:
            if not is_categorical(name, col_j):
                continue

            k, j = gen_freq(a, b, [col_i, col_j])
            kl = 1 / (1 + kl_div(k, j).sum())
            res.append([name, col_i, name, col_j, kl])

for table in tables:
    a, b = tables[table]
    cols_i = sorted(a.keys())
    parent = table

    while get_parent(parent):
        parent = get_parent(parent)

        c, d = tables[parent]
        a, b = a.join(c, rsuffix=parent), b.join(d, rsuffix=parent)
        cols_j = sorted(c.keys())

        for col_i in cols_i:
            if not is_categorical(table, col_i):
                continue

            for col_j in cols_j:
                if not is_categorical(parent, col_j):
                    continue

                k, j = gen_freq(a, b, [col_i, col_j])
                kl = 1 / (1 + kl_div(k, j).sum())
                res.append([table, col_i, parent, col_j, kl])


In [ ]:
idx_tuples = []
for table in reversed(tables):
    a, _ = tables[table]
    for col in sorted(a.keys()):
        if is_categorical(table, col):
            idx_tuples.append((table, col))

idx = pd.MultiIndex.from_tuples(idx_tuples, names=['table_i', 'col_i'])
cols = pd.MultiIndex.from_tuples(reversed(idx_tuples), names=['table_j', 'col_j'])

pt = pd.DataFrame(res, columns=["table_i", "col_i", "table_j", "col_j", "KL"])
pt = pt.pivot_table(
    values="KL", index=["table_i", "col_i"], columns=["table_j", "col_j"]
)
pt = pt.reindex(index=idx, columns=cols)
pt.style.background_gradient(axis=1).applymap(
    lambda x: "color: transparent; background-color: transparent"
    if pd.isnull(x)
    else ""
)